In [6]:
import os, sys, argparse, paramiko
from getpass import getpass
from os.path import exists

In [7]:
# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password:  ")

# Host information
host = "192.168.6.71"
port = 2222
username = "sarah.fornaldes"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication failed")

# TASK 2

In [8]:
sftp = ssh.open_sftp()

# Run a command to scan all known directories on Linux that have a *bin* directory
command = 'sudo -S /home/ubuntu/kraken --folder /usr/bin --folder --folder /usr/sbin/ --folder /usr/local/bin --folder /sbin --folder /usr/local/sbin --folder /bin'

ssh.exec_command('kraken')
sftp.put('kraken', 'kraken')

thePass = getpass(prompt="Please enter your SSH password:  ")

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

stdin.write(thePass + "\n")
stdin.flush()

# Get results  from stdout
lines =  stdout.readlines()
#print(lines)

# Convert the list to a string
output =  ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output = sepHeader + output + sepFooter

# Save the cmd_output to a file
with open('results123.txt', 'a') as f:
    f.write(cmd_output)

#ssh.close()
#sftp.close()

In [9]:
#sftp = ssh.open_sftp()

# Uploads results.txt file
uploadFiles = ['results123.txt']

# Runs the lsof command, lsof lists all open files for a process                
command = "sudo -S lsof -i -n | awk '{print $2}' "

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

# Input password to run the command as sudo, including a new line character that provides "Enter" as if you were typing it in on a keyboard
stdin.write(thePass + "\n")

# Get results  from stdout
lines =  stdout.readlines()

# Convert the list to a string
output =  ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output =  sepHeader + output +  sepFooter

# Save the cmd_output to a file
with open('resultsLsof.txt', 'a') as f:
    f.write(cmd_output)

# End SFTP Session
#ssh.close()
#sftp.close()

In [10]:
# https://stackoverflow.com/questions/24883009/how-to-compare-lists-whose-elements-are-regular-expressions-in-python 
list1 = ['519423', '561559']
list2 = ['593','603','603','830','855','855','935','935','561559','600020', '600170', '600390', '600489', '600523', '600626', '600716', '600818', '600854', '600947', '600952', '601052', '601067', '601169', '601242', '601357', '601383', '601485', '601534', '601633', '601653', '601752', '601759', '601861', '601881', '601977', '601982', '602084', '602108', '602204', '602220', '602318', '602330', '602429', '602470', '602568', '602604', '602705', '602900', '603028', '603036', '603134', '603284', '603390']
import re

results = ({i:j  for j in list1 for i in list2 if re.match(i,j)})
print(results)
 

{'561559': '561559'}


In [11]:
import re
matchPID = '561559'

#print stuff on eitjer side of PID
with open ('resultsLsof.txt', 'r') as f:
    for line in f:
        y = re.findall(r''+matchPID+'', line)
        print (line)


### BEGIN sudo -S lsof -i -n ###



COMMAND      PID            USER   FD   TYPE  DEVICE SIZE/OFF NODE NAME

systemd-n    593 systemd-network   19u  IPv4   19178      0t0  UDP 10.223.0.144:bootpc 

systemd-r    603 systemd-resolve   12u  IPv4   19238      0t0  UDP 127.0.0.53:domain 

systemd-r    603 systemd-resolve   13u  IPv4   19239      0t0  TCP 127.0.0.53:domain (LISTEN)

lwsmd        855            root   44u  IPv4 2302099      0t0  TCP 10.223.0.144:52820->192.168.4.4:ldap (ESTABLISHED)

sshd         935            root    3u  IPv4   24209      0t0  TCP *:ssh (LISTEN)

sshd         935            root    4u  IPv6   24221      0t0  TCP *:ssh (LISTEN)

sshd      561391            root    4u  IPv4 2256818      0t0  TCP 10.223.0.144:ssh->10.223.0.1:36544 (ESTABLISHED)

sshd      561490          ubuntu    4u  IPv4 2256818      0t0  TCP 10.223.0.144:ssh->10.223.0.1:36544 (ESTABLISHED)

egrep     561559        p.dalton    5u  IPv6 2257680      0t0  TCP *:http-alt (LISTEN)

sshd      566

Compare pids from kraken and cyber

use command = "sudo -S lsof -p matchPID | awk '{print $2}' "


#lsof -p xxx


results =[]

with open('krakenPIDS.txt', 'r') as f:
    for eachLine in f:
        results.append(eachLine)


    command = 'sudo -S lsof -p | 
